In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sasuke-2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5478010%2F9080073%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240801%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240801T083405Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9275a0ce33ed00124c0920e61ca9df04742cdc72cfb76b6f4157987717634ace1e617e1e65f05254b6040f367b511aa4c87f3be0a81eed7d4fff4e7f66db0dda7c9aebad40e1183d75e7c8406b4058690f6f1bf93098ee19879e0aab833385505d0a63a24366824ae43960354d009b1604595c23166618493785c27bd0ac1fdb6d3f3e9ac30c35d9ae246299c159499658ce67c18ff6f5c9a862f149023312ae406b58771a2d622f52156beed9d17a1ccc37ae395aac1d0de481af2861f1058788ce4a211017911994800f4735a4b59434f67145ca53ba7a3257a21cf8ddbd2475dd825f03ef25b22a082508696babed689d1820da4d00e40cefa5aa081efdb5,sasuke-1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5478014%2F9080077%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240801%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240801T083405Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dace5abb56fc3747bc89eb45de57c5d6e2198ccd161eba2080343b97b3ab69c33629d814a00bca30ca6314ac9aa87e0e054a11e5f8681f2a783e3f5c00516ffa7a64875fc3d72bdb0627b086bc5aa487e1e77526ac6b306105728dbb6ddb654a537c00026a6c8a0a500637505948c88843f24e0850a70d96926912e5fbcfa9f8e6b630e1e6011c0133b6ac5086e8d64d084a7def37c208ebab12ca26fd821a620c1d9e195daeb4cde9d3ee3ef5648e3e6dcb02e2fb585eef1e47575317cd3504369c85b669cd18bdf6650509cf0a2d2dca2341e573f713ac60fec9547ae902856ef412fa3c37d6aa9dd161b445905f757d3168aef8beb35fa43b6c85ef82c1f75,big-car:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5478239%2F9080358%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240801%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240801T083405Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daa9f264d05a398fd14e444b5d645e89481e09a7a32be6425fcdab197c28ac0bc70e4b05d94f439b575f1f9c9eb4d9fe889ebf024e68f8b829fa4bcd0d291fa6c9b1a73e157211264c39367aa4240bdda48d03b350e72998ed30c5da491ef72cc3ab53d75a2ea1fe1207034fffcc7dea3c9dd1cc0b7cb7dc284899e7c31989086bdbd9dea390c8d221e94ef1d2613f4b6dea79d9ff8dd95a4f6967b19e657d83c3b0c7fb672606a9486d6ab15fbdf64f33b6533b829693ffa4d3508cc15d477f811458b47a38335bd564b326ef81c8d86dd8662f1a2fc6d910631ebabdfcac09c25ae821728b6041c049fc340885e75f00b1608b5f90ef7a2441dccd317c46eeb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 194023 bytes downloaded
Downloaded and uncompressed: sasuke-2
[==================================================] 142455 bytes downloaded
Downloaded and uncompressed: sasuke-1
[==================================================] 681801 bytes downloaded
Downloaded and uncompressed: big-car
Data source import complete.


In [2]:
import cv2
import numpy as np
import torch
from PIL import Image
import tqdm as tqdm
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install -qq -U diffusers transformers ftfy accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 16.1 MB/s eta 0:00:00


In [4]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16,variant="fp16"
)
pipeline.enable_model_cpu_offload()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

...

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
def pad_resize(frame, target_width=1280, target_height=720):
    org_height, org_width = frame.shape[:2]
    scale = min(target_height / org_height, target_width / org_width)

    new_width = int(org_width * scale)
    new_height = int(org_height * scale)

    resized_frame = cv2.resize(frame, (new_width, new_height))
    new_frame = np.zeros((target_height, target_width, 3), dtype=np.uint8)

    y_offset = (target_height - new_height) // 2
    x_offset = (target_width - new_width) // 2
    new_frame[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_frame

    return new_frame, x_offset, new_width


In [6]:
def inpaint_frame(frame, mask, pipeline):
    # frame and mask are resized to 512x512 for inpainting
    frame_resized = cv2.resize(frame, (512, 512))
    mask_resized = cv2.resize(mask, (512, 512))

    frame_image = Image.fromarray(frame_resized)
    mask_image = Image.fromarray(mask_resized)

    try:
        inpainted_frame = pipeline(prompt=" ", image=frame_image, mask_image=mask_image).images[0]

        # Resizing inpainted frame back to original size
        inpainted_frame = inpainted_frame.resize((frame.shape[1], frame.shape[0]))
        return np.array(inpainted_frame)
    except Exception as e:
        print(f"Error in inpainting frame: {e}")
        return frame

In [7]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames


In [8]:
def reconstruct_video(frames, output_video_path, fps=2):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()

In [9]:
def converter(input_video, output_video, frame_rate_reduction=1):

    frames = extract_frames(input_video)

    inpainted_frames = []
    for i, frame in enumerate(frames):
        if i % frame_rate_reduction == 0:  # Process frames based on reduction
            padded_frame, start_x, new_width = pad_resize(frame)
            mask = np.zeros_like(padded_frame[:, :, 0])
            mask[:, :start_x] = 1
            mask[:, start_x + new_width:] = 1
            inpainted_frame = inpaint_frame(padded_frame, mask, pipeline)
            inpainted_frames.append(inpainted_frame)

    if not output_video.endswith('.mp4'):
        output_video += '.mp4'
    reconstruct_video(inpainted_frames, output_video)
    return output_video

In [11]:
converter("/kaggle/input/big-car/input_lion_1.mp4","/kaggle/working/output_lion_11.mp4")

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

'/kaggle/working/output_lion_11.mp4'

In [ ]:
FRAMES=extract_frames("/kaggle/input/big-car/input_lion_2.mp4")
fr=extract_frames("/kaggle/working/output_lion_2.mp4")
print(len(FRAMES))
print(len(fr))
for i in fr:
    print(i.shape)

###